In [1]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt
# from problems.ising_problem2 import IsingProblem2
from problems.ising_problem import IsingProblem
from problems.toy_problem import ToyProblem
from qiskit.providers.fake_provider import *
from bin.cost_function import cost_loc, calc_err, cost_global
from bin.hyperparameters import *

import time

In [ ]:
# # Import the library that you need in order to use your IBM account
# from qiskit_ibm_provider import IBMProvider
# from secrets import *


# try:
#     IBMProvider()
# except:
#     IBMProvider.save_account(token=IBM_token, overwrite=True)

In [ ]:
# provider = IBMProvider()
# provider.backends()

In [ ]:
def configured_backend():
    # backend = provider.get_backend("ibm_osaka")
    backend = FakeManila()
    # backend.options.update_options(...)
    return backend

In [ ]:
n_qubits = 3
error = 0.01
cond_num = 50

problem = ToyProblem(
    n_qubits=n_qubits,
)

In [ ]:
# error = 0.3 # paper uses 0.01 and lower but its not converging that fast
# cond_num = 10
# n_qubits = 3

# problem = IsingProblem(
#     n_qubits=n_qubits,
#     J=0.1,
#     cond_num=cond_num
# )

In [ ]:
len(problem.c)

In [ ]:
# print(problem.get_eta_zeta()) # looks accurate

In [ ]:
w = q_delta * np.random.randn(problem.param_shape, requires_grad=True)
# w = q_delta * np.random.randn(batch_size, problem.param_shape, requires_grad=True)

In [ ]:
# dev_mu = qml.device("default.qubit", wires=n_qubits+1)
# dev_gamma = qml.device("default.qubit", wires=n_qubits*2 + 1)
dev_mu = qml.device("qiskit.remote", wires=n_qubits+1, backend=configured_backend())

In [ ]:
from IPython.display import clear_output

opt = qml.GradientDescentOptimizer(eta)
# opt = qml.AdagradOptimizer(eta)
# opt = qml.AdamOptimizer(eta) # TODO: tune decay terms

cost_history = []

err = float("inf")
it = 1

In [ ]:
def cost_agg(w):
    c = cost_loc(problem, w, dev_mu)
    # c = cost_global(problem, w, dev_mu, dev_gamma)
    # return np.mean(c, axis=0)
    return c

In [ ]:
best_err = 1000
best_w = w

start = time.time()
# while err > error:
for it in range(steps):
    w, cost = opt.step_and_cost(cost_agg, w)
    # w, cost = opt.step_and_cost(lambda w: cost_loc(problem, w, dev_mu), w)

    err = calc_err(n_qubits, cost, cond_num)
    if err < best_err:
        best_err = err
        best_w = w
    
    clear_output(wait=True)

    print("Step {:3d}       Cost_L = {:9.7f} \t error = {:9.7f}".format(it, cost, err), flush=True)
    cost_history.append(cost)

    it += 1

print(f"Training time: {time.time() - start}s")

In [ ]:
best_err

### Optimization studies:
- each epoch is taking ~ 20s
- Each $\mu$ calculation takes ~0.05s but occasionally spikes to ~0.3s
- Each $\mu_{sum}$ has to loop over each combination of A_l for each qubit -> $c^2n$ operations -> ~15s
- Each $|\psi|$ also has to loop over each combination, but we're unable to recycle computation because these don't apply $CZ$ like the above -> ~4s
- The ising problem has 8 C's
    - Note that `len(c) = n_qubits * 2` by definition of H_ising
- 8 * 8 * 4 * 2 = ~512 mu accumulations

--> $8n^3 + 4n^2$ iterations

It should be possible to multithread the calculation, but not sure if that's feasible when testing on an actual QC


* n_qubits = 5: 1100 mu accumulations
* n_qubits = 6: 1872 mu accumulations
* n_qubits = 7: 2940 mu accumulations

In [ ]:
plt.style.use("seaborn")
plt.plot(np.log(cost_history), "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()

Qualitatively, it's converging slower than the toy problem, suggesting more iterations are needed

In [ ]:
from inference import get_cprobs, get_qprobs
c_probs = get_cprobs(problem)

dev_x = qml.device("qiskit.remote", wires=n_qubits, backend=configured_backend())
# dev_x = qml.device("default.qubit", wires=n_qubits, shots=n_shots)
q_probs = get_qprobs(problem, best_w, dev_x)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))

ax1.bar(np.arange(0, 2 ** n_qubits), c_probs, color="blue")
ax1.set_xlim(-0.5, 2 ** n_qubits - 0.5)
ax1.set_xlabel("Vector space basis")
ax1.set_title("Classical probabilities")

ax2.bar(np.arange(0, 2 ** n_qubits), q_probs, color="green")
ax2.set_xlim(-0.5, 2 ** n_qubits - 0.5)
ax2.set_xlabel("Hilbert space basis")
ax2.set_title("Quantum probabilities")

plt.show()